In [29]:
import pandas as pd
import os

In [30]:
Desktop = os.path.join(os.path.expanduser("~"), "Desktop")

filepath_root = os.path.join(Desktop, "LBP_Revision")
inclusion_root = os.path.join(filepath_root, "Inclusion_Codes") #inclusion code location
filtered_root = os.path.join(filepath_root, "Filtered_Files") #Filtered data location (Filtered data aggregated all diagnosis by patient)
unfiltered_root = os.path.join(filepath_root, "All_csv_files") #unfiltered data location
combined_root = os.path.join(filepath_root, "LBP_X_Files") #LBP + X filtered location


os.path.exists(filepath_root)

path_inclusion_crit_LBP = os.path.join(inclusion_root,'inclusion_codes_LBP.xlsx')
path_inclusion_crit_DJD = os.path.join(inclusion_root,'inclusion_codes_DJD.xlsx')
path_inclusion_crit_fracture = os.path.join(inclusion_root,'inclusion_codes_fracture.xlsx')
path_inclusion_crit_other = os.path.join(inclusion_root,'inclusion_codes_other.xlsx')
path_inclusion_crit_scoliosis = os.path.join(inclusion_root,'inclusion_codes_scoliosis.xlsx')

In [31]:
def get_cpts_fromfile(filepath=''):
    df_cpt = pd.read_excel(filepath, dtype=str, sheet_name='CPT Codes')
    return df_cpt['CPT Code']       

def get_icds_fromfile(filepath=''):        
    df_icd = pd.read_excel(filepath,dtype=str, sheet_name='ICD Codes')
    return df_icd['ICD Code']

In [32]:
LBP_filename = 'LBP_all'
fracture_filename = 'fracture_all'
LBP_all_data = pd.read_csv(unfiltered_root + '/' + LBP_filename + '.csv', encoding = 'utf8',dtype=str)
fracture_all_data = pd.read_csv(unfiltered_root + '/' + fracture_filename + '.csv', encoding = 'utf8',dtype=str)

In [37]:
#CPT Code
#LBP only
incl_cpts_LBP = get_cpts_fromfile(path_inclusion_crit_LBP)
incl_cpts_text_LBP = '|'.join(incl_cpts_LBP.str.strip())

#fracture
incl_cpts_fracture = get_cpts_fromfile(path_inclusion_crit_fracture)
incl_cpts_text_fracture = '|'.join(incl_cpts_fracture.str.strip())

#ICD Code
#LBP only
incl_icds_LBP = get_icds_fromfile(path_inclusion_crit_LBP)
incl_icds_text_LBP = '|'.join(incl_icds_LBP.str.strip())

#fracture
incl_icds_fracture = get_icds_fromfile(path_inclusion_crit_fracture)
incl_icds_text_fracture = '|'.join(incl_icds_fracture.str.strip())

In [33]:
print(LBP_all_data.keys())


Index(['EMPI', 'MGH_MRN', 'Date.prc', 'Date.dia', 'Code_Type.prc', 'Code.prc',
       'Code_Type.dia', 'Code.dia', 'Diagnosis_Name', 'Procedure_Name',
       'Gender_Legal_Sex', 'Age', 'Language', 'Race1', 'Marital_status',
       'Religion', 'Date_Of_Death', 'AgeAtPrc', 'Admit_Date', 'Discharge_Date',
       'Discharge_Disposition', 'diagnosis_list', 'Diagnosis_list_combined',
       'Diagnosis_list_no_NA', 'City', 'State', 'Zip', 'Country',
       'Insurance_1', 'Insurance_2', 'Insurance_3', 'Phone_Code',
       'Physical_Date', 'BMI', 'Height', 'Weight', 'Smoking', 'Alcohol_Use',
       'Helper'],
      dtype='object')


In [34]:
LBP_diagcode = LBP_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
LBP_diagcode['Diagnosis_combined'] = str(0)
LBP_diagcode['LBP?'] = str(0)
print(LBP_diagcode.keys())

LBP_diagcode_reference = LBP_diagcode



Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'LBP?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18707/3317639091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18707/3317639091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['LBP?'] = str(0)


In [35]:
LBP_diagcode.loc[0,'Diagnosis_combined'] = LBP_diagcode.loc[0,'Code.dia']
for i in range(1,len(LBP_diagcode)):
    if (LBP_diagcode.loc[i,'EMPI'] == LBP_diagcode.loc[i-1,'EMPI']):
        if LBP_diagcode.loc[i,'Code.dia'] != LBP_diagcode.loc[i-1,'Code.dia']:
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia'] + ', ' + str(LBP_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia']
        continue;

In [36]:
LBP_filtered = LBP_diagcode[LBP_diagcode['Helper'].notna()]
LBP_filtered_save = LBP_filtered

In [38]:
temp_df_LBP = pd.DataFrame
temp_df_LBP = LBP_filtered['Diagnosis_combined'].str.contains(incl_icds_text_LBP, regex=True)

LBP_filtered.index = range(len(LBP_filtered.index))

for i in range(len(temp_df_LBP)):
    if temp_df_LBP.iloc[i] == True:
        LBP_filtered.loc[i,'LBP?'] = 'LBP'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18707/484879454.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_filtered.loc[i,'LBP?'] = 'LBP'


In [39]:
fracture_diagcode = fracture_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
fracture_diagcode['Diagnosis_combined'] = str(0)
fracture_diagcode['fracture?'] = str(0)
print(fracture_diagcode.keys())

fracture_diagcode_reference = fracture_diagcode

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'fracture?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18707/1215614615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fracture_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18707/1215614615.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fracture_diagcode['fracture?'] = str(0)


In [40]:
fracture_diagcode.loc[0,'Diagnosis_combined'] = fracture_diagcode.loc[0,'Code.dia']
for i in range(1,len(fracture_diagcode)):
    if (fracture_diagcode.loc[i,'EMPI'] == fracture_diagcode.loc[i-1,'EMPI']):
        if fracture_diagcode.loc[i,'Code.dia'] != fracture_diagcode.loc[i-1,'Code.dia']:
            fracture_diagcode.loc[i,'Diagnosis_combined'] = fracture_diagcode.loc[i,'Code.dia'] + ', ' + str(fracture_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            fracture_diagcode.loc[i,'Diagnosis_combined'] = fracture_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        fracture_diagcode.loc[i,'Diagnosis_combined'] = fracture_diagcode.loc[i,'Code.dia']
        continue;

In [41]:
fracture_filtered = fracture_diagcode[fracture_diagcode['Helper'].notna()]
fracture_filtered_save = fracture_filtered

In [42]:
temp_df_fracture = pd.DataFrame
temp_df_fracture = fracture_filtered['Diagnosis_combined'].str.contains(incl_icds_text_fracture, regex=True)

fracture_filtered.index = range(len(fracture_filtered.index))

for i in range(len(temp_df_fracture)):
    if temp_df_fracture.iloc[i] == True:
        fracture_filtered.loc[i,'fracture?'] = 'fracture'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18707/747291113.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fracture_filtered.loc[i,'fracture?'] = 'fracture'


In [43]:
LBP_filtered.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?
0,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP
1,100004276,92,Other,21.15,72100,M54.9,useful,M54.9,LBP
2,100006144,78,Black,23.33,72100,G89.29,useful,G89.29,LBP
3,100006348,80,White,27,72040,G89.29,useful,G89.29,LBP
4,100007295,70,White,NaN,74000,G89.29,useful,G89.29,LBP


In [44]:
fracture_filtered.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,fracture?
0,100000243,92,Declined,24.73,72040,S22.000D,useful,S22.000D,fracture
1,100002085,89,White,27.92,72100,M48.56XA,useful,M48.56XA,fracture
2,100004276,92,Other,21.15,72100,S22.030A,useful,S22.030A,fracture
3,100007095,83,White,20.57,72100,M48.56XA,useful,M48.56XA,fracture
4,100010573,85,Other,21.06,72100,S32.000A,useful,S32.000A,fracture


In [45]:
LBP_and_fracture = pd.concat([LBP_filtered,fracture_filtered])

In [46]:
LBP_and_fracture.index = range(len(LBP_and_fracture.index))
LBP_and_fracture['Helper'] = str(0)

In [47]:
LBP_and_fracture.sort_values(by=['EMPI'], inplace=True, ignore_index=True)
LBP_and_fracture_copy = LBP_and_fracture
LBP_and_fracture.head(10)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,fracture?
0,100000152,81,White,27.02,72100,G89.29,0,G89.29,LBP,NaN
1,100000243,92,Declined,24.73,72040,M54.9,0,M54.9,LBP,NaN
2,100000243,92,Declined,24.73,72040,S22.000D,0,S22.000D,NaN,fracture
3,100001350,84,Black,26.92,72040,S22.018B,0,S22.018B,NaN,fracture
4,100001760,88,White,NaN,72040,G89.29,0,G89.29,LBP,NaN
5,100001760,88,White,NaN,72040,M48.54XA,0,M48.54XA,NaN,fracture
6,100002085,89,White,27.92,72100,M48.56XA,0,M48.56XA,NaN,fracture
7,100002524,98,White,29.76,72100,S32.010A,0,S32.010A,NaN,fracture
8,100003506,96,Declined,20.06,72100,M54.9,0,M54.9,LBP,NaN
9,100004276,92,Other,17.51,72040,S22.080A,0,S22.080A,NaN,fracture


In [48]:
for i in range(1,len(LBP_and_fracture)):
    if LBP_and_fracture.loc[i,'EMPI'] == LBP_and_fracture.loc[i-1,'EMPI']:
        if LBP_and_fracture.loc[i,'Diagnosis_combined'] != LBP_and_fracture.loc[i-1,'Diagnosis_combined']:
            LBP_and_fracture.loc[i,'Diagnosis_combined'] = LBP_and_fracture.loc[i,'Diagnosis_combined'] + ', ' + str(LBP_and_fracture.loc[i-1,'Diagnosis_combined'])
        if LBP_and_fracture.loc[i,'LBP?'] != 'LBP':
            if LBP_and_fracture.loc[i-1,'LBP?'] == 'LBP':
                LBP_and_fracture.loc[i,'LBP?'] = 'LBP'
        if LBP_and_fracture.loc[i,'fracture?'] != 'fracture':
            if LBP_and_fracture.loc[i-1,'fracture?'] == 'fracture':
                LBP_and_fracture.loc[i,'fracture?'] = 'fracture'
    else:
        LBP_and_fracture.loc[i-1,'Helper'] = 'useful'

In [49]:
LBPnfracture_filtered = LBP_and_fracture.loc[LBP_and_fracture['Helper'] == 'useful']
LBPnfracture_filtered.reset_index(drop =True, inplace=True)
LBPnfracture_filtered['Both'] = str(0)

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18707/3989062123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBPnfracture_filtered['Both'] = str(0)


In [50]:
for i in range(0,len(LBPnfracture_filtered)):
    if LBPnfracture_filtered.loc[i,'LBP?'] == 'LBP':
        if LBPnfracture_filtered.loc[i,'fracture?'] == 'fracture':
            LBPnfracture_filtered.loc[i,'Both'] = 'Both'

In [51]:
LBPnfracture_filtered.head(20)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,fracture?,Both
0,100000152,81,White,27.02,72100,G89.29,useful,G89.29,LBP,NaN,0
1,100000243,92,Declined,24.73,72040,S22.000D,useful,"S22.000D, M54.9",LBP,fracture,Both
2,100001350,84,Black,26.92,72040,S22.018B,useful,S22.018B,NaN,fracture,0
3,100001760,88,White,NaN,72040,M48.54XA,useful,"M48.54XA, G89.29",LBP,fracture,Both
4,100002085,89,White,27.92,72100,M48.56XA,useful,M48.56XA,NaN,fracture,0
5,100002524,98,White,29.76,72100,S32.010A,useful,S32.010A,NaN,fracture,0
6,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP,NaN,0
7,100004276,92,Other,21.15,72100,M54.9,useful,"M54.9, G89.29, S22.030A, S22.080A",LBP,fracture,Both
8,100004591,78,White,NaN,72040,M48.56XA,useful,M48.56XA,NaN,fracture,0
9,100004610,98,White,NaN,72100,M48.55XA,useful,"M48.55XA, S30.0XXA",LBP,fracture,Both


In [52]:
LBP_only = LBPnfracture_filtered.loc[LBPnfracture_filtered['LBP?'] == 'LBP']
LBP_only = LBP_only[LBP_only['Both'] != 'Both']
fracture_only = LBPnfracture_filtered.loc[LBPnfracture_filtered['fracture?'] == 'fracture']
fracture_only = fracture_only[fracture_only['Both'] != 'Both']
LBPfracture_Both_only = LBPnfracture_filtered.loc[LBPnfracture_filtered['Both'] == 'Both']

In [53]:
LBP_and_fracture_total = pd.concat([LBP_diagcode,fracture_diagcode])
LBP_and_fracture_total.drop_duplicates(subset=['EMPI'],inplace=True)
len(LBP_and_fracture_total)

6043

In [54]:
print('Number of Total Patients(LBP+fracture):  ' + str(len(LBP_and_fracture_total)-1))
print('Number of LBP only:  ' + str(len(LBP_only)))
print('Number of fracture only:  ' + str(len(fracture_only)))
print('Number of Both only:  ' + str(len(LBPfracture_Both_only)))

#LBP_filtered = LBP
#fracture_filtered = fracture
#LBP & fracture filtered = LBPnfracture_filtered


Number of Total Patients(LBP+fracture):  6042
Number of LBP only:  1951
Number of fracture only:  1722
Number of Both only:  2369


In [55]:
LBP_patients = LBP_filtered.sort_values(by=['EMPI'], ignore_index=True)
LBP_patients.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?
0,100000152,81,White,27.02,72100,G89.29,useful,G89.29,LBP
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP
3,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP
4,100004276,92,Other,17.51,72040,G89.29,useful,G89.29,LBP


In [56]:
fracture_csv_path = os.path.join(combined_root,'LBP_fracture_patients.csv')
LBPfracture_Both_only.to_csv(fracture_csv_path)

In [57]:
fracture_only_csv_path = os.path.join(filtered_root,'fracture_patients.csv')
fracture_filtered.to_csv(fracture_only_csv_path)

In [58]:
LBPfracture_Both_only.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,fracture?,Both
1,100000243,92,Declined,24.73,72040,S22.000D,useful,"S22.000D, M54.9",LBP,fracture,Both
3,100001760,88,White,NaN,72040,M48.54XA,useful,"M48.54XA, G89.29",LBP,fracture,Both
7,100004276,92,Other,21.15,72100,M54.9,useful,"M54.9, G89.29, S22.030A, S22.080A",LBP,fracture,Both
9,100004610,98,White,NaN,72100,M48.55XA,useful,"M48.55XA, S30.0XXA",LBP,fracture,Both
11,100006144,78,Black,23.33,72100,G89.29,useful,"G89.29, M54.9, S32.009K",LBP,fracture,Both
